In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point

Geocode and mapping tutorial

- https://towardsdatascience.com/pythons-geocoding-convert-a-list-of-addresses-into-a-map-f522ef513fd6#5352

How to hide API keys in Python

- https://medium.com/black-tech-diva/hide-your-api-keys-7635e181a06c

And in general

- https://stackoverflow.com/questions/44342276/how-to-push-code-to-github-hiding-the-api-keys

In [2]:
%store -r google_maps_API_Key

geolocator = GoogleV3(api_key=google_maps_API_Key)

In [7]:
### In this cell, I need to create my DF ###

data = {
    'address':['4535 Nautilus Court, Miami Beach, FL','2035 North Meridian Avenue, Miami Beach, FL','2211 Meridian Avenue, Miami Beach, FL','254 Palm Avenue, Miami Beach, FL'],
    'buyer':['Bryant Yunker Jr. and Nancy Yunker','Allon and Allison Bloch','Zachi and Ekaterina Wainstein','David Gebbia'],
    'seller':['Orlandus Andre Branch III','2035 Meridian LLC','Sabal Development',''],
    'sale_price_s':['$6.3M','$5M','$5.5M'],
    'sale_price_int':[6300000,5000000,5500000]
}

df = pd.DataFrame(data)

In [8]:
df['loc'] = df['address'].apply(geolocator.geocode, timeout=10)
df['loc']

0    (4535 Nautilus Ct, Miami Beach, FL 33140, USA,...
1    (2035 Meridian Ave, Miami Beach, FL 33139, USA...
2    (2211 Meridian Ave, Miami Beach, FL 33139, USA...
Name: loc, dtype: object

In [9]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

In [10]:
df

,address,buyer,seller,sale_price_s,sale_price_int,loc,point,lat,lon,altitude
0,"4535 Nautilus Court, Miami Beach, FL",Bryant Yunker Jr. and Nancy Yunker,Orlandus Andre Branch III,$6.3M,6300000,"(4535 Nautilus Ct, Miami Beach, FL 33140, USA,...","(25.8201721, -80.13459329999999, 0.0)",25.820172,-80.134593,0.0
1,"2035 North Meridian Avenue, Miami Beach, FL",Allon and Allison Bloch,2035 Meridian LLC,$5M,5000000,"(2035 Meridian Ave, Miami Beach, FL 33139, USA...","(25.7976922, -80.13559649999999, 0.0)",25.797692,-80.135596,0.0
2,"2211 Meridian Avenue, Miami Beach, FL",Zachi and Ekaterina Wainstein,Sabal Development,$5.5M,5500000,"(2211 Meridian Ave, Miami Beach, FL 33139, USA...","(25.7996423, -80.1346805, 0.0)",25.799642,-80.134681,0.0


In [11]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=14)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format('Non-waterfront Miami Beach home sales over $5M')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('Pre-pandemic, a non-waterfront home sale in Miami Beach over $5M was unheard of.')



marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["sale_price_s"], r["seller"], r['buyer'], r['address'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Buyer: {bold_start}{info[2]}{bold_end}{new_line}Seller: {bold_start}{info[1]}{bold_end}{new_line}Address: {bold_start}{info[3]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
m

In [12]:
m.save('index.html')